In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# read data
train_data_df = pd.read_csv('/kaggle/input/titanic/train.csv')

In [3]:
# check data
train_data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# check shape
train_data_df.shape

(891, 12)

In [5]:
# check for null values
train_data_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
# impute age with median
train_data_df['Age'].fillna(train_data_df['Age'].median(),inplace=True)

In [7]:
# check for null values
train_data_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
# check cabin
train_data_df['Cabin'].value_counts

<bound method IndexOpsMixin.value_counts of 0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object>

In [9]:
# drop cabin column
train_data_df.drop('Cabin',axis=1,inplace=True)

In [10]:
# drop name column, ticket column
train_data_df.drop(["Name","Ticket"],axis=1,inplace=True)

In [11]:
# check sex column
train_data_df.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [12]:
# make male = 1, female = 0
train_data_df['Sex'] = train_data_df['Sex'].map({'male': 1, 'female': 0})

In [13]:
# check df
train_data_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,S
1,2,1,1,0,38.0,1,0,71.2833,C
2,3,1,3,0,26.0,0,0,7.9250,S
3,4,1,1,0,35.0,1,0,53.1000,S
4,5,0,3,1,35.0,0,0,8.0500,S


In [14]:
# convert age as int
train_data_df['Age'] = train_data_df['Age'].astype(int)

In [15]:
# check embarked column
train_data_df.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [16]:
# one hot encode Embarked column and drop it from the original df
encoded_df = pd.get_dummies(train_data_df['Embarked'], prefix='Embarked')
df_encoded = pd.concat([train_data_df, encoded_df], axis=1)
df_encoded = df_encoded.drop('Embarked', axis=1)

In [17]:
# Splitting the data into features and target variable
X = df_encoded.drop('Survived', axis=1)  # Features
y = df_encoded['Survived']  # Target variable

In [18]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
import tensorflow as tf

# Build the ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
23/23 [==============================] - 2s 14ms/step - loss: 0.5973 - accuracy: 0.7022 - val_loss: 0.5013 - val_accuracy: 0.7877
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4720 - accuracy: 0.7949 - val_loss: 0.4219 - val_accuracy: 0.8101
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4393 - accuracy: 0.8160 - val_loss: 0.4100 - val_accuracy: 0.8156
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4022 - accuracy: 0.8357 - val_loss: 0.4115 - val_accuracy: 0.8268
Epoch 5/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4040 - accuracy: 0.8315 - val_loss: 0.4157 - val_accuracy: 0.8324
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 0.3842 - accuracy: 0.8441 - val_loss: 0.4246 - val_accuracy: 0.8380
Epoch 7/100
23/23 [==============================] - 0s 4ms/step - loss: 0.3691 - accuracy: 0.8413 - val_loss: 0.4191 - val_accuracy: 0.8268
Epoch 8/100


In [23]:
# predict the test data with new model
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

In [24]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [25]:
# applying the same pre-processing steps
# impute age with median
test_df['Age'].fillna(test_df['Age'].median(),inplace=True)

# drop cabin column
test_df.drop('Cabin',axis=1,inplace=True)

# drop name column, ticket column
test_df.drop(["Name","Ticket"],axis=1,inplace=True)

# make male = 1, female = 0
test_df['Sex'] = test_df['Sex'].map({'male': 1, 'female': 0})

# convert age as int
test_df['Age'] = test_df['Age'].astype(int)

# one hot encode Embarked column and drop it from the original df
encoded_df = pd.get_dummies(test_df['Embarked'], prefix='Embarked')
df_encoded = pd.concat([test_df, encoded_df], axis=1)
df_encoded = df_encoded.drop('Embarked', axis=1)

df_encoded = scaler.transform(df_encoded)

In [26]:
df_encoded

array([[ 1.72973532,  0.81303367,  0.7243102 , ..., -0.46146201,
         3.29646275, -1.67555562],
       [ 1.73363318,  0.81303367, -1.38062393, ..., -0.46146201,
        -0.30335547,  0.59681695],
       [ 1.73753104, -0.40055118,  0.7243102 , ..., -0.46146201,
         3.29646275, -1.67555562],
       ...,
       [ 3.34734697,  0.81303367,  0.7243102 , ..., -0.46146201,
        -0.30335547,  0.59681695],
       [ 3.35124483,  0.81303367,  0.7243102 , ..., -0.46146201,
        -0.30335547,  0.59681695],
       [ 3.35514269,  0.81303367,  0.7243102 , ...,  2.16702561,
        -0.30335547, -1.67555562]])

In [27]:
y_pred = model.predict(df_encoded)

14/14 [==============================] - 0s 2ms/step


In [28]:
y_pred.shape

(418, 1)

In [29]:
threshold = 0.5
y_pred_classes = (y_pred > threshold).astype(int)

In [30]:
y_pred_classes.shape

(418, 1)

In [31]:
final_df = pd.DataFrame()

In [32]:
final_df['PassengerId'] = test_df.PassengerId
final_df['Survived'] = y_pred_classes

In [33]:
final_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [34]:
# write to csv file
# Assuming your DataFrame is called df
final_df.to_csv('submission.csv', index=False)